In [0]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
ROOT='/content/drive/My Drive/dl/推荐评论/'
BERT_ROOT='/content/drive/My Drive/dl/google-quest-challenge/pretrained-bert-models-for-pytorch/新建文件夹/bert-base-chinese/'
BERT_TOK_ROOT='/content/drive/My Drive/dl/google-quest-challenge/pretrained-bert-models-for-pytorch/bert-base-chinese-vocab.txt'
MAX_LEN=22

In [0]:
!pip install pytorch_pretrained_bert
!pip install transformers
!pip install scikit-learn==0.21.3
import sklearn
sklearn.__version__

'0.21.3'

In [0]:
import sklearn
import torch
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from pytorch_pretrained_bert import BertModel, BertTokenizer
root='/content/drive/My Drive/google-quest-challenge/'
import numpy as np
import time
import tqdm

import math
import scipy
import torch.nn.functional as F
import random
import transformers, sys, os, gc
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,
)
from torch.utils.data import DataLoader, Dataset,RandomSampler, SequentialSampler
from transformers.modeling_bert import BertPreTrainedModel 
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold
from tqdm import tqdm_notebook
from sklearn.metrics import roc_curve 
import numpy as np
import collections


In [0]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True
setup_seed(42)

In [0]:
labels,train_data=[],[]
test=[]
weight=[]
lenth = []

In [0]:
with open (ROOT+'data/train.txt', 'r') as f:
  for line in f:
    label,text = line.split('\t')
    labels.append(int(label))
    train_data.append(text.replace('\n',''))

In [0]:
counter = torch.load(ROOT+'counter.pkl')

In [0]:
pre_count=[]

In [0]:
for line in train_data:
  pre = []
  for word in line:
    pre.append(counter.get(word,0))
  pre = pre +(MAX_LEN-len(pre))*[0]
  pre_count.append(pre)

In [0]:
pre_count = np.array(pre_count)


In [0]:
for label in labels:
  if label == 1:
    weight.append(3)
  else:
    weight.append(1)


In [0]:
tokenizer=BertTokenizer.from_pretrained(BERT_TOK_ROOT)
train_idx = []
train_mask = []
train_segment = []

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [0]:
for line in range(len(train_data)):
  token = tokenizer.tokenize(train_data[line])
  token = ["[CLS]"] + token + ["[SEP]"]
  lenth.append(len(token))
  if len(token)> MAX_LEN:
      raise IndexError("Token length more than max seq length!")

  mask = [1]*len(token)+ [0]*(MAX_LEN-len(token))
  sent = [0]*MAX_LEN
  idx = tokenizer.convert_tokens_to_ids(token)
  idx = idx + (MAX_LEN-len(idx))*[0]
  train_idx.append(idx)
  train_mask.append(mask)
  train_segment.append(sent)

# **统计特征**

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD


In [0]:
text_encoder = Pipeline([
    ('Text-TF-IDF', TfidfVectorizer(ngram_range=(1, 3))),
    ('Text-SVD', TruncatedSVD(n_components = 600))], verbose=True)

In [0]:
text_encoder=torch.load(ROOT+'text_encoder.pkl')

In [0]:
tfidf=text_encoder.transform(train_data)

In [0]:
lenths = np.array(lenth)
lenths=lenths.reshape(-1,1)

In [0]:
use = lenths

# **Dataset**

In [0]:
class EvalDataset(Dataset):
  def __init__(self,train_idx,train_mask,train_segment,use,label=None):
    self.train_idx = torch.tensor(train_idx,dtype=torch.int).long()
    self.train_mask = torch.tensor(train_mask,dtype=torch.int).long()
    self.train_segment = torch.tensor(train_segment,dtype=torch.int).long()
    self.use = torch.tensor(use , dtype=torch.float32)
    self.label = None
    if label is not None:
      self.label = torch.tensor(label,dtype=torch.float32)
  def __len__(self):
    return len(self.train_idx)
  def __getitem__(self,item):
    if self.label is not None:

      output = {
        'train_idx' : self.train_idx[item],
        'train_mask' : self.train_mask[item],
        'train_segment' :self.train_segment[item],
        'use':self.use[item],
        'label' : self.label[item]
    }
    else:
      output = {
        'train_idx' : self.train_idx[item],
        'train_mask' : self.train_mask[item],
        'use' : self.use[item],
        'train_segment' :self.train_segment[item],
      }
    return {key: value for key, value in output.items()}

In [0]:
def get_ids(x,y):
  z=[]
  for i in y:
    z.append(x[i])
  return z

In [0]:

## Stolen from transformer code base without any noble intention.

## Stolen from transformer code base without any noble intention.
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1)
        nn.init.xavier_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / torch.sum(a, 1, keepdim=True) + 1e-10
        
        weighted_input = x.permute(1,0,2) * torch.unsqueeze(a, -1)
        return torch.sum(weighted_input, 1)
class cnnbert(BertPreTrainedModel):

    def __init__(self, config,kernel_sizes,num_channels,use_shape):
        super(cnnbert, self).__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.3)
        self.bn = nn.LayerNorm(768)
        self.gelu=GELU()
        self.att1=Attention(512,21)
        self.att2=Attention(512,20)
        self.att3=Attention(512,19)
        self.att4=Attention(512,18)
        self.att5=Attention(512,13)
        self.softmax=nn.Softmax(dim=1)
        self.pre_cnn=nn.Linear(2*sum(num_channels)+use_shape,1)
        self.linear = nn.Linear(2*sum(num_channels)+use_shape,2*sum(num_channels)+use_shape)

        self.ln_cnn=nn.LayerNorm(sum(num_channels))
        self.wt=torch.nn.Parameter(torch.rand((13,1),dtype=torch.float32,device=torch.device('cuda'),requires_grad=True))
        nn.init.xavier_normal_(self.wt)
        self.pool = GlobalAVGPool1d()
        self.convs = nn.ModuleList()
        for c, k in zip(num_channels, kernel_sizes):
          self.convs.append(nn.Conv1d(in_channels = 768,
          out_channels = c,
          kernel_size = k))

    def forward(
        self,
        use,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        
    ):

        outputs = self.bert(
           
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        w1=outputs[2] 
        batch_size1,len_seq1,e_s1=w1[1].size()
        bert_output1=w1[0].to(device)
        bert_output1=bert_output1.unsqueeze(1)
        for i in range(1,13):
          bert_output1=torch.cat((bert_output1,w1[i].to(device).unsqueeze(1)),dim=1)
        bert_output1=bert_output1.view(batch_size1,13,-1)
        bert_output1=bert_output1.permute(0,2,1)
        bert_output1=torch.matmul(bert_output1,((self.wt)))
        bert_output1=bert_output1.view(batch_size1,len_seq1,e_s1)#下可接CNN或者att
        bert_output1=self.bn(bert_output1)
        bert_output1=bert_output1.permute(0, 2, 1)

        bert_output1 = self.dropout(bert_output1)
        conout1=self.att1(self.gelu(self.convs[0](bert_output1).permute(2,0,1)))
        conout2=self.att2(self.gelu(self.convs[1](bert_output1).permute(2,0,1)))
        conout3=self.att3(self.gelu(self.convs[2](bert_output1).permute(2,0,1)))
        conout4=self.att4(self.gelu(self.convs[3](bert_output1).permute(2,0,1)))
        conout5=self.att5(self.gelu(self.convs[4](bert_output1).permute(2,0,1)))
        conout=torch.cat((conout1,conout2,conout3,conout4,conout5),dim=1)
        encoding =torch.cat([self.pool(self.gelu(conv(bert_output1))).squeeze(-1) for conv in self.convs], dim=1)
       
        encoding=torch.cat((encoding,conout,use),dim=1)
       
        logits = self.pre_cnn((encoding))
        return logits 

# **用于生成伪标签的模型**

In [0]:
'''class cnnbert(BertPreTrainedModel):

    def __init__(self, config,kernel_sizes,num_channels,use_shape):
        super(cnnbert, self).__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.3)
        self.bn = nn.LayerNorm(768)
        self.gelu=GELU()
        self.softmax=nn.Softmax(dim=1)
        self.pre_cnn=nn.Linear(1*sum(num_channels)+use_shape,1)
        self.linear = nn.Linear(1*sum(num_channels)+use_shape,1*sum(num_channels)+use_shape)

        self.ln_cnn=nn.LayerNorm(1*sum(num_channels))
        self.wt=torch.nn.Parameter(torch.rand((13,1),dtype=torch.float32,device=torch.device('cuda'),requires_grad=True))
        nn.init.xavier_normal_(self.wt)
        self.pool = GlobalAVGPool1d()
        self.convs = nn.ModuleList()
        for c, k in zip(num_channels, kernel_sizes):
          self.convs.append(nn.Conv1d(in_channels = 768,
          out_channels = c,
          kernel_size = k))

    def forward(
        self,
        use,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        
    ):

        outputs = self.bert(
           
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        w1=outputs[2] 
        batch_size1,len_seq1,e_s1=w1[1].size()
        bert_output1=w1[0].to(device)
        bert_output1=bert_output1.unsqueeze(1)
        for i in range(1,13):
          bert_output1=torch.cat((bert_output1,w1[i].to(device).unsqueeze(1)),dim=1)
        bert_output1=bert_output1.view(batch_size1,13,-1)
        bert_output1=bert_output1.permute(0,2,1)
        bert_output1=torch.matmul(bert_output1,self.softmax((self.wt)))
        bert_output1=bert_output1.view(batch_size1,len_seq1,e_s1)#下可接CNN或者att
        bert_output1=self.bn(bert_output1)
        bert_output1=bert_output1.permute(0, 2, 1)

        bert_output1 = self.dropout(bert_output1)
        encoding =torch.cat([self.pool(self.gelu(conv(bert_output1))).squeeze(-1) for conv in self.convs], dim=1)
                            
        encoding=torch.cat((encoding,use),dim=1)
        
        logits = self.pre_cnn((encoding))
        return logits '''

In [0]:
class GlobalAVGPool1d(nn.Module):
    def __init__(self):
        super(GlobalAVGPool1d, self).__init__()
    def forward(self, x):
         # x shape: (batch_size, channel, seq_len)
        return F.avg_pool1d(x, kernel_size=x.shape[2]) # shape: (batch_size, channel, 1)

In [0]:
class GELU(nn.Module):
    """
    Paper Section 3.4, last paragraph notice that BERT used the GELU instead of RELU
    """

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

In [0]:
bert_model_config = BERT_ROOT+'/bert_config.json'
bert_config = BertConfig.from_json_file(bert_model_config)
bert_config.num_labels = 1
bert_config.output_hidden_states=True
bert_config

BertConfig {
  "architectures": null,
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_hidden_layers": 12,
  "num_labels": 1,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": true,
  "output_past": true,
  "pad_token_id": null,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",


In [0]:
def my_loss(input,target,weight=None):
  input=torch.sigmoid(input)
  if weight is not None:
    loss =torch._C._nn.binary_cross_entropy(input,target)
    loss*=weight
    return loss.sum()
  else:
    loss = torch._C._nn.binary_cross_entropy(input,target)
    return loss.sum()

In [0]:
def train_model(train_loader ,train_shape, optimizer ,scheduler, criterion, config=None):
  model.train()
  acc = 0
  auc = 0
  avg_loss = 0
  optimizer.zero_grad()
  for idx, batch in tqdm.tqdm(enumerate(train_loader),mininterval=2,desc='--Training',leave=False):
    input_ids = batch['train_idx'].to(device)
    input_mask = batch['train_mask'].to(device)
    input_segments = batch['train_segment'].to(device)
    label = batch['label'].to(device).view(-1)
    use = batch['use'].to(device)
    output_train = model(use=use,input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_mask,
                             token_type_ids = input_segments,)
    loss = criterion(output_train.view(-1),label)                      
    #loss = my_loss(output_train.view(-1),label,weight)
    avg_loss += loss.item()
    loss.backward()
    optimizer.step()
    scheduler.step()
    acc += ((torch.sigmoid(output_train.cpu().view(-1))>0.5).int()==label.cpu()).sum()
    #print(len(np.unique(label.cpu().int())))
    #auc += sklearn.metrics.roc_auc_score( label.cpu(),torch.sigmoid(output_train.view(-1)).detach().cpu().numpy())

    '''fgm.attack()
    output_train = model(use,input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_mask,
                             token_type_ids = input_segments,)
    loss_adv = criterion(output_train.view(-1),label)     
    loss_adv.backward()
    fgm.restore()

    optimizer.step()
    scheduler.step()
    '''
    
    ema.update()
    optimizer.zero_grad()
    
  return avg_loss/train_shape,acc.item()/train_shape,auc/len(train_loader)

In [0]:
def valid_model(valid_loader ,val_shape,  criterion, batch_size ,config=None):
  model.eval()
  acc = 0
  auc = 0
  avg_loss = 0
  pred = np.zeros((val_shape))
  ema.apply_shadow()
  with torch.no_grad():
    for idx, batch in tqdm.tqdm(enumerate(valid_loader),mininterval=2,desc='--Valing',leave=False):
      input_ids = batch['train_idx'].to(device)
      input_mask = batch['train_mask'].to(device)
      input_segments = batch['train_segment'].to(device)
      label = batch['label'].to(device).view(-1)
      use = batch['use'].to(device)
      output_train = model(use=use,input_ids = input_ids.long(),
                              labels = None,
                              attention_mask = input_mask,
                              token_type_ids = input_segments,
                              )
      loss = criterion(output_train.view(-1),label)
      #loss = my_loss(output_train.view(-1),label,weight)
      pred[idx*batch_size : (idx+1)*batch_size] = output_train.view(-1).detach().cpu().squeeze().numpy()
      avg_loss += loss.item()
      acc += ((torch.sigmoid(output_train.cpu().view(-1))>0.5).int()==label.cpu()).sum()
      if input_ids.size()[0]!=1:

        auc += sklearn.metrics.roc_auc_score( (label.cpu().int()),torch.sigmoid(output_train.view(-1)).detach().cpu().numpy())
  ema.restore()
  return avg_loss/val_shape,acc.item()/val_shape,auc/len(valid_loader),torch.sigmoid(torch.tensor(pred)).numpy()

In [0]:
class EMA():
    def __init__(self, model, decay):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()

    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data
                param.data = self.shadow[name]

    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

In [0]:
class FGM():
  def __init__(self,model):
    self.model=model
    self.backup={}
  def attack(self,epsilon=0.1,emb_name='word_emb'):
    for name,param in self.model.named_parameters():
      if param.requires_grad and emb_name in name:
        self.backup[name]=param.data.clone()
        if (param.grad)== None:
          continue
        norm=torch.norm(param.grad)
        if norm!= 0 and not torch.isnan(norm):
          r_at=epsilon * param.grad/norm
          param.data.add_(r_at)
  def restore(self,emb_name='word_emb'):
    for name,param in self.model.named_parameters():
      if param.requires_grad and emb_name in name:
        assert name in self.backup
        param.data=self.backup[name]
    self.backup={}

In [0]:
class cycleOptimn():
    def __init__(self,base_lr,max_lr,optimizer,stepsize):
        self.optimizer=optimizer
        self.base_lr=base_lr
        self.max_lr=max_lr
        self.stepsize=stepsize
        self.current_step=0
    def zero_grad(self):
        self.optimizer.zero_grad()
    def step_and_update_lr(self):
        self._update_learning_rate()
        self.optimizer.step()
    def _update_learning_rate(self):
        self.current_step+=1

        lr=self.get_triangular_lr(self.current_step,self.stepsize,self.base_lr,self.max_lr)
        for param_group in  self.optimizer.param_groups:
                param_group['lr']=lr
       
    def get_triangular_lr(self,iteration, stepsize, base_lr, max_lr):

        cycle = np.floor(1 + iteration/(2  * stepsize))
        x = np.abs(iteration/stepsize - 2 * cycle + 1)
        lr = base_lr + (max_lr - base_lr) * np.maximum(0, (1-x))
        return lr


In [0]:
BATCH_SIZE =70
NUM_EPOCH=5
device = torch.device('cuda')

In [0]:
torch.cuda.get_device_name()

'Tesla P100-PCIE-16GB'

In [0]:
floder = StratifiedKFold(n_splits=5,random_state=0,shuffle=False)
predict = np.zeros(len(train_idx))


In [0]:
 
 
def transfer_model(pretrained_file, model):
    '''
    只导入pretrained_file部分模型参数
    tensor([-0.7119,  0.0688, -1.7247, -1.7182, -1.2161, -0.7323, -2.1065, -0.5433,-1.5893, -0.5562]
    update:
        D.update([E, ]**F) -> None.  Update D from dict/iterable E and F.
        If E is present and has a .keys() method, then does:  for k in E: D[k] = E[k]
        If E is present and lacks a .keys() method, then does:  for k, v in E: D[k] = v
        In either case, this is followed by: for k in F:  D[k] = F[k]
    :param pretrained_file:
    :param model:
    :return:
    '''
    pretrained_dict = torch.load(pretrained_file)  # get pretrained dict
    model_dict = model.state_dict()  # get model dict
    # 在合并前(update),需要去除pretrained_dict一些不需要的参数
    pretrained_dict = transfer_state_dict(pretrained_dict, model_dict)
    model_dict.update(pretrained_dict)  # 更新(合并)模型的参数
    model.load_state_dict(model_dict)
    return model
 
 
def transfer_state_dict(pretrained_dict, model_dict):
    '''
    根据model_dict,去除pretrained_dict一些不需要的参数,以便迁移到新的网络
    url: https://blog.csdn.net/qq_34914551/article/details/87871134
    :param pretrained_dict:
    :param model_dict:
    :return:
    '''
    # state_dict2 = {k: v for k, v in save_model.items() if k in model_dict.keys()}
    state_dict = {}
    for k, v in pretrained_dict.items():
        if k in model_dict.keys():
            # state_dict.setdefault(k, v)
            state_dict[k] = v
        else:
            print("Missing key(s) in state_dict :{}".format(k))
    return state_dict
 
 


In [0]:
wlabel_label=torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_label_bin.pkl')
wlabel_train_idx = torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_idx.pkl')
wlabel_train_mask = torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_mask.pkl')
wlabel_train_segment = torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_segment.pkl')
wlabel_train_use=torch.load('/content/drive/My Drive/dl/推荐评论/CNN/w20_train_lenth.pkl')

In [0]:
use_shape = use.shape[1]
use = list(use)

In [0]:
for fold,(train, test) in enumerate(floder.split(train_idx,labels)):
  if fold == 3:
    continue 
  print(fold)
  t_idx = get_ids(train_idx , train)+wlabel_train_idx
  t_mask = get_ids(train_mask , train)+wlabel_train_mask
  t_segment = get_ids(train_segment , train)+wlabel_train_segment
  t_weight = get_ids(weight , train)
  #t_label = get_ids(labels , train)# (np.hstack((np.array(get_ids(labels , train)),wlabel_label)))
  t_label = (np.vstack((np.array(get_ids(labels , train)).reshape(-1,1),wlabel_label.reshape(-1,1))))

  t_use = (np.vstack((np.array(get_ids(use , train)),(wlabel_train_use).reshape(-1,1))))
  #t_use=(np.array(get_ids(use , train)))
  valid_idx = get_ids(train_idx , test)
  valid_mask = get_ids(train_mask , test)
  valid_segment = get_ids(train_segment , test)
  valid_weight = get_ids(weight , test)
  valid_label = get_ids(labels , test)
  valid_use = get_ids(use , test)
  train_shape = len(t_idx)
  valid_shape = len(valid_idx)
  train_set = EvalDataset(t_idx , t_mask , t_segment , t_use ,t_label)
  vla_set = EvalDataset(valid_idx , valid_mask , valid_segment ,valid_use ,valid_label)
  train_loader = DataLoader (train_set , batch_size=BATCH_SIZE,shuffle=True)
  valid_loader = DataLoader (vla_set , batch_size=BATCH_SIZE,shuffle=False)
  kernel_sizes,nums_channels = [2,3,4,5,10], [512,512,512,512,512]
  model = cnnbert.from_pretrained(BERT_ROOT,kernel_sizes=kernel_sizes,num_channels=nums_channels,use_shape=use_shape,config = bert_config)
  #model = transfer_model('/content/drive/My Drive/dl/美团/CLS/best_param_score_1',model)
  model.zero_grad();
  model.to(device);
  #fgm = FGM(model)
  ema=EMA(model,0.995)
  ema.register()
  def is_backbone(n):
        return "bert" in n



  

  optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005, eps=4e-5,weight_decay=0.8)
  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0.05, num_training_steps= 5*len(train_loader))
  #optimizer=cycleOptimn(0.00001,0.0001,torch.optim.AdamW(model.parameters(), lr=0.00002, eps=4e-5,weight_decay=0.8),90)
  criterion = nn.BCEWithLogitsLoss()
  best_avg_loss = -100
  best_score = -1
  best_param_score = None
  for epoch in range(NUM_EPOCH):
    torch.cuda.empty_cache()
    start_time = time.time()
    train_loss,train_acc,train_auc=train_model(train_loader ,train_shape, optimizer , scheduler, criterion, config=None)
    val_loss,val_acc,val_auc,pred=valid_model(valid_loader ,valid_shape,  criterion, BATCH_SIZE ,config=None)
    elapsed_time = time.time() - start_time
    print('epoch{}/{} , train_loss: {} ,train_acc:{} ,train_auc: {} \n \n val_loss :{} , val_acc :{} , val_auc:{} ,time: {}\n'.format(epoch+1,NUM_EPOCH,train_loss,train_acc,train_auc,val_loss,val_acc,val_auc,elapsed_time))
    if best_score < val_auc:
      best_score = val_auc
      best_param_score = model.state_dict()
      predict[test] = pred
      torch.save(best_param_score,'/content/drive/My Drive/dl/推荐评论/0.96840版本基础 伪标签/CNN_attack0.1/best_param_score{}.pkl'.format(fold+1))
      torch.save(ema,'/content/drive/My Drive/dl/推荐评论/0.96840版本基础 伪标签/CNN_attack0.1/EMA{}.pkl'.format(fold+1))

  del model, optimizer, criterion, scheduler
  torch.cuda.empty_cache()
  del valid_loader, train_loader, vla_set, train_set

0


epoch1/5 , train_loss: 0.0020144295440213175 ,train_acc:0.9409238776837996 ,train_auc: 0.0 
 
 val_loss :0.0029912285415949727 , val_acc :0.9075288972196188 , val_auc:0.9723019447566703 ,time: 501.6054196357727



/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type cnnbert. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type GELU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Attention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type GlobalAVGPool1d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
--Training: 515it [04

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
sklearn.metrics.roc_auc_score(labels,predict)

In [0]:
0.9712348313293575

+att 0.96822

无att: 0.9684   lr:0.00005

+预训练: 不行

+use 